In [17]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
import openpyxl as openpyxl
import random 
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [20]:
import os
os.getcwd()

'c:\\Users\\uffeb\\OneDrive - Aarhus universitet\\Kandidat\\Master\\Speciale\\Speciale---Stibo'

In [21]:
file_path = r"C:\Users\uffeb\OneDrive - Aarhus universitet\Kandidat\Master\Speciale\Speciale---Stibo\Data\WA_Fn-UseC_-Telco-Customer-Churn.csv"
#file_path = r"C:\Users\josep\OneDrive - Aarhus universitet\Speciale\Speciale---Stibo\Data\WA_Fn-UseC_-Telco-Customer-Churn.csv"

In [22]:
data = pd.read_csv(file_path)

In [23]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [24]:
# Checking the data types of all the columns
data.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [25]:
# Converting Total Charges to a numerical data type.
data.TotalCharges = pd.to_numeric(data.TotalCharges, errors='coerce')
data.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [26]:
#Removing missing values 
data.dropna(inplace = True)

#Remove customer IDs from the data set
df2 = data.iloc[:,1:]

#Convertin the predictor variable in a binary numeric variable
df2['Churn'].replace(to_replace='Yes', value=1, inplace=True)
df2['Churn'].replace(to_replace='No',  value=0, inplace=True)

C:\Users\uffeb\AppData\Local\Temp\ipykernel_21220\3726204027.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['Churn'].replace(to_replace='Yes', value=1, inplace=True)
C:\Users\uffeb\AppData\Local\Temp\ipykernel_21220\3726204027.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [27]:
# Set the random seed
random.seed(2137)
np.random.seed(2137)

# Assuming df2 is your original DataFrame
y = df2['Churn']
X = df2.drop(['Churn'], axis=1)

# Get the numeric and categorical column names
objecttypes_num = X.select_dtypes(include=['float64', 'int64']).columns
objecttypes_cat = X.select_dtypes(include=['object']).columns

# Transformer for numerical columns
transformer_num = Pipeline([
    ('scaler', StandardScaler())
])

# Transformer for categorical columns
transformer_cat = Pipeline([
    ('encoder', OneHotEncoder(drop='first'))
])

# Preprocessor for all features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_num, objecttypes_num),
        ('cat', transformer_cat, objecttypes_cat)
    ], remainder='passthrough'
)

# Standardize and one-hot encode the features
X_processed = preprocessor.fit_transform(X)


# Get the column names after transformation
processed_colnames = preprocessor.get_feature_names_out()
processed_colnames = [name.replace('num__', '').replace('cat__', '') for name in processed_colnames]

# Convert the processed features to a DataFrame
df_processed = pd.DataFrame(X_processed, columns=processed_colnames)

# Concatenate 'churn' variable to the DataFrame
df_processed['Churn'] = y.reset_index(drop=True)

In [28]:
df_processed.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,-0.440327,-1.280248,-1.161694,-0.994194,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
1,-0.440327,0.064303,-0.260878,-0.173740,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
2,-0.440327,-1.239504,-0.363923,-0.959649,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
3,-0.440327,0.512486,-0.747850,-0.195248,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
4,-0.440327,-1.239504,0.196178,-0.940457,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1


In [29]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_processed.drop('Churn', axis=1), df_processed['Churn'], test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier (you can choose any other classifier)
classifier = RandomForestClassifier(random_state=42)

# Train the classifier on the training set
classifier.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_output = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', classification_report_output)


Accuracy: 0.7846481876332623
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.90      0.86      1033
           1       0.62      0.48      0.54       374

    accuracy                           0.78      1407
   macro avg       0.72      0.69      0.70      1407
weighted avg       0.77      0.78      0.77      1407



## Model explainability - Local and Global interpretations

In [30]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [31]:
import shap

# Create an explainer for the best model
explainer_rf = shap.Explainer(y_pred)

# Calculate SHAP values for a single prediction (change the index as needed)
shap_values_rf = explainer_rf(X_test.iloc[0])

# Calculate SHAP values for a set of predictions
shap_values_rf = explainer_rf(X_test)

shap.summary_plot(shap_values_rf, X_test)

TypeError: The passed model is not callable and cannot be analyzed directly with the given masker! Model: [0 0 1 ... 0 0 0]